# Installs

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 16.8 MB/s eta 0:00:00


# Imports

In [2]:
import pickle

In [3]:
import numpy as np
import pandas as pd

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, matthews_corrcoef, precision_score, recall_score, f1_score


# Load data

In [5]:
with open('vm_c_rate_diff_if.pickle', 'rb') as handle:
   marked_vm_series = pickle.load(handle)

In [6]:
def get_supervised_dataset(marked_series, memory_steps=10, future_step=1):
    _, data, labels = marked_series
    X, y = [], []
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data.reshape(-1, 1))
    for i in range(len(data) - memory_steps + 1 - future_step):
        X.append(data[i:(i + memory_steps)])
        y.append(labels[i + memory_steps - 1 + future_step])
    return np.array(X), np.array(y).astype('int')

# Metric

In [7]:
def compute_results(y_true, y_pred):
    metrics = dict()
    metrics['Accuracy'] = accuracy_score(y_true, y_pred)
    metrics['Precision'] = precision_score(y_true, y_pred)
    metrics['Recall'] = recall_score(y_true, y_pred)
    metrics['F1-score'] = f1_score(y_true, y_pred)
    metrics['Balanced Accuracy'] = balanced_accuracy_score(y_true, y_pred)
    # metrics["Mathew's Correlation Coefficient"] = \
    #                                        matthews_corrcoef(y_true, y_pred)
    error_matrix = pd.DataFrame(
        confusion_matrix(y_true, y_pred, labels=[1, 0]),
        columns=['a(x) = 1', 'a(x) = 0'],
        index=['y = 1', 'y = 0'],
    ).to_dict()
    return metrics, error_matrix

# Final predict

## Boosting

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import StackingClassifier

In [ ]:
def fit_predict_xgbc(colour, memory_steps=50, future_step=1):
    X, y = get_supervised_dataset(marked_vm_series[colour], memory_steps=memory_steps, future_step=future_step)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)
    params = {
        'min_child_weight': [1, 2, 5, 10],
        'gamma': [0.2, 0.5, 1],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 8]
        }
    xgbc = XGBClassifier(
        objective='binary:logistic',
        random_state=42,
        scale_pos_weight=49,
        learning_rate=0.2,
        n_estimators=500,
        verbosity=0,
        nthread=1
    )
    folds = 5
    param_comb = 5
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)
    random_search = RandomizedSearchCV(xgbc, param_distributions=params,
                                   n_iter=param_comb, scoring='f1',
                                   n_jobs=4, cv=skf.split(X_train, y_train),
                                   verbose=0, random_state=42 )
    random_search.fit(X_train, y_train)
    bxgbc = random_search.best_estimator_
    y_pred = bxgbc.predict(X_test)
    classifier_metrics, error_matrix = compute_results(y_test, y_pred)
    return bxgbc, y_pred, classifier_metrics, error_matrix

In [ ]:
def fit_predict_lgbmc(colour, memory_steps=50, future_step=1):
    X, y = get_supervised_dataset(marked_vm_series[colour], memory_steps=memory_steps, future_step=future_step)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)
    params = {
        'learning_rate': [0.1, 0.2, 0.3],
        'num_leaves': [6,8,12,16],
        'boosting_type' : ['gbdt', 'dart'],
        'colsample_bytree' : [0.64, 0.65, 0.66],
        'subsample' : [0.7, 0.725, 0.75],
        'max_depth': [-1, 3, 5, 8],
        'reg_alpha' : [1,1.2],
        'reg_lambda' : [1,1.2,1.4],
    }
    lgbmc = LGBMClassifier(
        verbose=-1,
        objective='binary',
        scale_pos_weight=50,
        min_gain_to_split=1,
        learning_rate=0.2,
        n_estimators=500,
        max_depth=5,
        random_state=42
    )
    folds = 5
    param_comb = 5
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)
    random_search = RandomizedSearchCV(lgbmc, param_distributions=params,
                                   n_iter=param_comb, scoring='f1',
                                   n_jobs=4, cv=skf.split(X_train, y_train),
                                   verbose=0, random_state=42 )
    random_search.fit(X_train, y_train)
    blgbmc = random_search.best_estimator_
    y_pred = blgbmc.predict(X_test)
    classifier_metrics, error_matrix = compute_results(y_test, y_pred)
    return blgbmc, y_pred, classifier_metrics, error_matrix

In [ ]:
def fit_predict_cbc(colour, memory_steps=50, future_step=1):
    X, y = get_supervised_dataset(marked_vm_series[colour], memory_steps=memory_steps, future_step=future_step)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)
    # params = {'learning_rate': [0.1, 0.2],'depth': [4, 6, 8, 10],'auto_class_weights': ['Balanced'],'l2_leaf_reg': [0, 5, 7],'random_seed': [42]}
    cbc = CatBoostClassifier(
        auto_class_weights = 'Balanced',
        learning_rate=0.2,
        depth=6,
        l2_leaf_reg=7,
        task_type='GPU',
        verbose=False,
        random_seed=42
    )
    #folds = 5
    #param_comb = 5
    #skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)
    #random_search = RandomizedSearchCV(cbc, param_distributions=params,n_iter=param_comb, scoring='f1',n_jobs=4, cv=skf.split(X_train, y_train),verbose=0, random_state=42)
    #random_search.fit(X_train, y_train)
    #bcbc = random_search.best_estimator_
    bcbc = cbc
    bcbc.fit(X_train, y_train)
    y_pred = bcbc.predict(X_test)
    classifier_metrics, error_matrix = compute_results(y_test, y_pred)
    return bcbc, y_pred, classifier_metrics, error_matrix

In [ ]:
def fit_predict_cbc(colour, memory_steps=50, future_step=1):
    X, y = get_supervised_dataset(marked_vm_series[colour], memory_steps=memory_steps, future_step=future_step)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)
    params = {'learning_rate': [0.1, 0.2],
              'depth': [4, 6, 8],
              'auto_class_weights': ['Balanced'],
              'l2_leaf_reg': [0, 5, 7],
              'random_seed': [42]}
    cbc = CatBoostClassifier(
        #auto_class_weights = 'Balanced',
        #learning_rate=0.2,
        #depth=6,
        #l2_leaf_reg=7,
        #task_type='GPU',
        thread_count = 4,
        verbose=False,
        num_trees=500,
        eval_metric='F1'
        #random_seed=42
    )
    folds = 5
    param_comb = 5
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)
    random_search = RandomizedSearchCV(cbc, param_distributions=params,n_iter=param_comb, scoring='f1',n_jobs=4, cv=skf.split(X_train, y_train),verbose=0, random_state=42)
    random_search.fit(X_train, y_train)
    bcbc = random_search.best_estimator_
    #bcbc = cbc
    bcbc.fit(X_train, y_train)
    y_pred = bcbc.predict(X_test)
    classifier_metrics, error_matrix = compute_results(y_test, y_pred)
    return bcbc, y_pred, classifier_metrics, error_matrix

In [ ]:
def fit_predict_stack_logreg(estimators, colour, memory_steps=50, future_step=1):
    X, y = get_supervised_dataset(marked_vm_series[colour], memory_steps=memory_steps, future_step=future_step)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)
    named_estimators = [(name, estimator) for name, estimator in zip([f'{i}' for i in range(len(estimators))], estimators)]
    clf = StackingClassifier(estimators=named_estimators,
                             final_estimator=LogisticRegressionCV(random_state=42, class_weight='balanced', cv=5, scoring='f1'))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    classifier_metrics, error_matrix = compute_results(y_test, y_pred)
    return clf, y_pred, classifier_metrics, error_matrix

In [ ]:
def all_predictions(colour, memory_steps=50, future_steps=1):
    classifiers_metrics = dict()
    classifiers_error_matrix = dict()
    print(f'Colour: {colour}')
    xgbc, _, classifier_metrics, error_matrix = fit_predict_xgbc(colour, memory_steps, future_steps)
    classifiers_metrics['xgbc'] = classifier_metrics
    classifiers_error_matrix['xgbc'] = error_matrix
    print('Done xgbc')
    lgbmc, _, classifier_metrics, error_matrix = fit_predict_lgbmc(colour, memory_steps, future_steps)
    classifiers_metrics['lgbmc'] = classifier_metrics
    classifiers_error_matrix['lgbmc'] = error_matrix
    print('Done lgbm')
    cbc, _, classifier_metrics, error_matrix = fit_predict_cbc(colour, memory_steps, future_steps)
    classifiers_metrics['cbc'] = classifier_metrics
    classifiers_error_matrix['cbc'] = error_matrix
    print('Done cbc')
    stackc, _,  classifier_metrics, error_matrix = fit_predict_stack_logreg([xgbc, lgbmc, cbc], colour, memory_steps, future_steps)
    classifiers_metrics['stackc'] = classifier_metrics
    classifiers_error_matrix['cbc'] = error_matrix
    print('Done stackc')
    return classifiers_metrics, classifiers_error_matrix

In [ ]:
# c, e = all_predictions('brown', memory_steps=50, future_steps=1)

In [ ]:
# pd.DataFrame(c)

In [ ]:
colour_classifier_metrics = dict()
colour_confusion_matrix = dict()

In [ ]:
# for colour in ['brown', 'pink', 'white', 'blue', 'violet']:
  #   colour_classifier_metrics[colour], colour_confusion_matrix[colour] = all_predictions(colour, memory_steps=50, future_steps=1)

In [ ]:
#pd.DataFrame.from_dict({(i,j): colour_classifier_metrics[i][j]
#                           for i in colour_classifier_metrics.keys()
#                           for j in colour_classifier_metrics[i].keys()},
#                       orient='index').transpose()

In [ ]:
future_steps = [1, 2, 3, 4, 5]

In [ ]:
for steps in future_steps:
    print(f'Steps = {steps}')
    colour_classifier_metrics[steps] = dict()
    colour_confusion_matrix[steps] = dict()
    for colour in ['brown', 'pink', 'white', 'blue', 'violet']:
        colour_classifier_metrics[steps][colour], colour_confusion_matrix[steps][colour] = \
            all_predictions(colour, memory_steps=50, future_steps=steps)

Steps = 1
Colour: brown
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: pink
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: white
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: blue
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: violet
Done xgbc
Done lgbm
Done cbc
Done stackc
Steps = 2
Colour: brown
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: pink
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: white
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: blue
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: violet
Done xgbc
Done lgbm
Done cbc
Done stackc
Steps = 3
Colour: brown
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: pink
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: white
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: blue
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: violet
Done xgbc
Done lgbm
Done cbc
Done stackc
Steps = 4
Colour: brown
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: pink
Done xgbc
Done lgbm
Done cbc
Done stackc
Colour: white
Done xgbc
Done 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Done xgbc
Done lgbm
Done cbc
Done stackc


In [ ]:
with open('colour_classifier_metrics_steps_cv.pickle', 'wb') as handle:
   pickle.dump(colour_classifier_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('colour_confusion_matrix_steps_cv..pickle', 'wb') as handle:
   pickle.dump(colour_confusion_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def display_metrics(steps):
    return pd.DataFrame.from_dict({(i,j): colour_classifier_metrics[steps][i][j]
                           for i in colour_classifier_metrics[steps].keys()
                           for j in colour_classifier_metrics[steps][i].keys()},
                       orient='index').transpose()

In [ ]:
display_metrics(1)

brown                                    pink            \
                       xgbc     lgbmc       cbc    stackc      xgbc     lgbmc   
Accuracy           0.983928  0.975892  0.979910  0.945756  0.974887  0.968860   
Precision          0.681818  0.409091  0.500000  0.216667  0.210526  0.129032   
Recall             0.375000  0.450000  0.400000  0.650000  0.102564  0.102564   
F1-score           0.483871  0.428571  0.444444  0.325000  0.137931  0.114286   
Balanced Accuracy  0.685706  0.718337  0.695900  0.800910  0.547440  0.544366   

                                          white                                \
                        cbc    stackc      xgbc     lgbmc       cbc    stackc   
Accuracy           0.979407  0.800100  0.976394  0.911100  0.974385  0.601708   
Precision          0.333333  0.047859  0.000000  0.079755  0.133333  0.032298   
Recall             0.051282  0.487179  0.000000  0.325000  0.050000  0.650000   
F1-score           0.088889  0.087156  0.000000  0.128079  0.072727  0.061538   
Balanced Accuracy  0.524616  0.646766  0.498206  0.624058  0.521668  0.625359   

                       blue                                  violet            \
                       xgbc     lgbmc       cbc    stackc      xgbc     lgbmc   
Accuracy           0.970367  0.903064  0.975892  0.763435  0.977398  0.907584   
Precision          0.047619  0.030675  0.000000  0.022173  0.000000  0.026316   
Recall             0.025000  0.125000  0.000000  0.250000  0.000000  0.100000   
F1-score           0.032787  0.049261  0.000000  0.040733  0.000000  0.041667   
Balanced Accuracy  0.507374  0.522008  0.497950  0.511981  0.498719  0.512071   

                                       
                        cbc    stackc  
Accuracy           0.978905  0.834756  
Precision          0.000000  0.035370  
Recall             0.000000  0.275000  
F1-score           0.000000  0.062678  
Balanced Accuracy  0.499487  0.560616

In [ ]:
display_metrics(3)

brown                                    pink            \
                       xgbc     lgbmc       cbc    stackc      xgbc     lgbmc   
Accuracy           0.978392  0.970854  0.979397  0.951759  0.972864  0.970352   
Precision          0.451613  0.312500  0.484848  0.230769  0.058824  0.083333   
Recall             0.350000  0.375000  0.400000  0.600000  0.025641  0.051282   
F1-score           0.394366  0.340909  0.438356  0.333333  0.035714  0.063492   
Balanced Accuracy  0.670641  0.679038  0.695641  0.779487  0.508720  0.520003   

                                          white                                \
                        cbc    stackc      xgbc     lgbmc       cbc    stackc   
Accuracy           0.977889  0.805025  0.976382  0.959799  0.974372  0.808543   
Precision          0.272727  0.051414  0.000000  0.045455  0.000000  0.045333   
Recall             0.076923  0.512821  0.000000  0.050000  0.000000  0.425000   
F1-score           0.120000  0.093458  0.000000  0.047619  0.000000  0.081928   
Balanced Accuracy  0.536411  0.661843  0.498205  0.514231  0.497179  0.620705   

                       blue                                  violet            \
                       xgbc     lgbmc       cbc    stackc      xgbc     lgbmc   
Accuracy           0.974874  0.907538  0.970352  0.714573  0.977889  0.896985   
Precision          0.083333  0.026316  0.000000  0.018248  0.000000  0.011834   
Recall             0.025000  0.100000  0.000000  0.250000  0.000000  0.050000   
F1-score           0.038462  0.041667  0.000000  0.034014  0.000000  0.019139   
Balanced Accuracy  0.509679  0.512051  0.495128  0.487051  0.498974  0.482179   

                                       
                        cbc    stackc  
Accuracy           0.971357  0.512060  
Precision          0.000000  0.022564  
Recall             0.000000  0.550000  
F1-score           0.000000  0.043350  
Balanced Accuracy  0.495641  0.530641

In [ ]:
display_metrics(5)

brown                                    pink            \
                       xgbc     lgbmc       cbc    stackc      xgbc     lgbmc   
Accuracy           0.980905  0.970352  0.977387  0.951759  0.976382  0.972864   
Precision          0.535714  0.306122  0.424242  0.214286  0.214286  0.173913   
Recall             0.375000  0.375000  0.350000  0.525000  0.076923  0.102564   
F1-score           0.441176  0.337079  0.383562  0.304348  0.113208  0.129032   
Balanced Accuracy  0.684167  0.678782  0.670128  0.742756  0.535642  0.546413   

                                          white                                \
                        cbc    stackc      xgbc     lgbmc       cbc    stackc   
Accuracy           0.979397  0.876884  0.968342  0.962814  0.977889  0.720603   
Precision          0.375000  0.070833  0.000000  0.027778  0.000000  0.034296   
Recall             0.076923  0.435897  0.000000  0.025000  0.000000  0.475000   
F1-score           0.127660  0.121864  0.000000  0.026316  0.000000  0.063973   
Balanced Accuracy  0.537180  0.660799  0.494103  0.503526  0.498974  0.600321   

                       blue                                  violet            \
                       xgbc     lgbmc       cbc    stackc      xgbc     lgbmc   
Accuracy           0.973869  0.900000  0.971859  0.630151  0.979899  0.891960   
Precision          0.071429  0.023952  0.000000  0.024590  0.000000  0.016575   
Recall             0.025000  0.100000  0.000000  0.450000  0.000000  0.075000   
F1-score           0.037037  0.038647  0.000000  0.046632  0.000000  0.027149   
Balanced Accuracy  0.509167  0.508205  0.495897  0.541923  0.500000  0.491859   

                                       
                        cbc    stackc  
Accuracy           0.976884  0.468342  
Precision          0.000000  0.024299  
Recall             0.000000  0.650000  
F1-score           0.000000  0.046847  
Balanced Accuracy  0.498462  0.557308

In [ ]:
for i in range(1, 6):
    (display_metrics(i)).to_csv(f'f{i}.csv')

## LSTM

In [8]:
X, y = get_supervised_dataset(marked_vm_series['brown'], memory_steps=50, future_step=1)
y = y.astype('float32')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify = y_train, random_state=42)

In [10]:
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], 1), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(30, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(20),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [12]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

In [13]:
class_weights

array([ 0.51025641, 24.875     ])

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', keras.metrics.Recall()])

In [15]:
model.fit(X_train, y_train, validation_data=(X_val, y_val),
          epochs=100, batch_size=32,
          class_weight={0: class_weights[0], 1: class_weights[1]}
          #class_weight={0:1, 1:48}
          )

Epoch 1/100
199/199 [==============================] - 9s 16ms/step - loss: 0.6952 - accuracy: 0.5905 - recall: 0.3594 - val_loss: 0.6906 - val_accuracy: 0.6300 - val_recall: 0.1562
Epoch 2/100
199/199 [==============================] - 2s 10ms/step - loss: 0.6943 - accuracy: 0.4728 - recall: 0.4453 - val_loss: 0.7031 - val_accuracy: 0.0201 - val_recall: 1.0000
Epoch 3/100
199/199 [==============================] - 2s 10ms/step - loss: 0.6942 - accuracy: 0.8037 - recall: 0.1875 - val_loss: 0.7163 - val_accuracy: 0.0201 - val_recall: 1.0000
Epoch 4/100
199/199 [==============================] - 2s 10ms/step - loss: 0.6944 - accuracy: 0.3246 - recall: 0.6562 - val_loss: 0.6959 - val_accuracy: 0.3455 - val_recall: 0.3750
Epoch 5/100
199/199 [==============================] - 2s 11ms/step - loss: 0.6942 - accuracy: 0.4573 - recall: 0.4609 - val_loss: 0.7069 - val_accuracy: 0.0201 - val_recall: 1.0000
Epoch 6/100
199/199 [==============================] - 2s 10ms/step - loss: 0.6938 - accur

In [16]:
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

63/63 [==============================] - 1s 5ms/step


Model overfits!!

In [17]:
c, e = compute_results(y_test, y_pred_binary)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
c

{'Accuracy': 0.9799095931692616,
 'Precision': 0.0,
 'Recall': 0.0,
 'F1-score': 0.0,
 'Balanced Accuracy': 0.5}

In [19]:
pd.DataFrame(e)

,a(x) = 1,a(x) = 0
y = 1,0,40
y = 0,0,1951
